# Infer-4-Bayesian-Networks : Reseaux Bayesiens Classiques

**Serie** : Programmation Probabiliste avec Infer.NET (4/13)  
**Duree estimee** : 55 minutes  
**Prerequis** : Infer-3-Factor-Graphs

---

## Objectifs

- Comprendre les reseaux bayesiens et leur structure
- Implementer le reseau classique Wet Grass / Sprinkler / Rain
- Maitriser les tables de probabilites conditionnelles (CPT)
- Distinguer inference causale et observationnelle
- Comprendre la D-separation et l'independance conditionnelle

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-3-Factor-Graphs](Infer-3-Factor-Graphs.ipynb) | [Infer-5-Skills-IRT](Infer-5-Skills-IRT.ipynb) |

---

## 1. Configuration

Cette section initialise l'environnement Infer.NET avec les packages necessaires pour construire et inferer des reseaux bayesiens. Les reseaux bayesiens sont des modeles graphiques diriges qui representent les relations de dependance conditionnelle entre variables.

In [1]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET pret !


In [2]:
// Chargement du helper pour visualiser les graphes de facteurs
#load "FactorGraphHelper.cs"

Console.WriteLine($"FactorGraphHelper charge. Graphviz disponible: {FactorGraphHelper.IsGraphvizAvailable()}");

FactorGraphHelper charge. Graphviz disponible: True


**Environnement pret.** Les packages Infer.NET charges incluent :
- `Microsoft.ML.Probabilistic` : moteur d'inference
- `Microsoft.ML.Probabilistic.Compiler` : compilation de modeles vers code C#
- `Microsoft.ML.Probabilistic.Models` : API de construction de modeles

Infer.NET utilise l'algorithme **Expectation Propagation** (EP) par defaut, particulierement adapte aux reseaux bayesiens avec variables discretes et continues.

## 2. Introduction aux Reseaux Bayesiens

### Definition

Un **reseau bayesien** est un graphe dirige acyclique (DAG) ou :
- Les **noeuds** representent des variables aleatoires
- Les **arcs** representent des dependances directes
- Chaque noeud a une **table de probabilite conditionnelle** (CPT)

### Proprietes

$$P(X_1, ..., X_n) = \prod_{i=1}^{n} P(X_i | \text{Parents}(X_i))$$

### Avantages

| Avantage | Description |
|----------|-------------|
| **Compacite** | Representation factorisee de la jointe |
| **Interpretabilite** | Structure causale explicite |
| **Inference efficace** | Algorithmes de propagation |
| **Apprentissage** | Structure et parametres apprenables |

## 3. Le Reseau Wet Grass / Sprinkler / Rain

### Structure

```
         (Cloudy)
         /      \
        v        v
   (Sprinkler)  (Rain)
        \        /
         v      v
        (WetGrass)
```

### Semantique

- **Cloudy** : Temps nuageux (cause commune)
- **Sprinkler** : Arroseur automatique (s'active moins si nuageux)
- **Rain** : Pluie (plus probable si nuageux)
- **WetGrass** : Herbe mouillee (effet commun)

### Tables de Probabilites Conditionnelles

| Variable | CPT |
|----------|-----|
| Cloudy | P(C=T) = 0.5 |
| Sprinkler | P(S=T\|C=T) = 0.1, P(S=T\|C=F) = 0.5 |
| Rain | P(R=T\|C=T) = 0.8, P(R=T\|C=F) = 0.2 |
| WetGrass | P(W=T\|S,R) - voir table complete |

In [3]:
// Implementation du reseau Wet Grass

// Variable racine : Cloudy
Variable<bool> cloudy = Variable.Bernoulli(0.5).Named("cloudy");

// Sprinkler conditionne par Cloudy
Variable<bool> sprinkler = Variable.New<bool>().Named("sprinkler");
using (Variable.If(cloudy))
{
    sprinkler.SetTo(Variable.Bernoulli(0.1));  // Peu probable si nuageux
}
using (Variable.IfNot(cloudy))
{
    sprinkler.SetTo(Variable.Bernoulli(0.5));  // Plus probable si ensoleille
}

// Rain conditionne par Cloudy
Variable<bool> rain = Variable.New<bool>().Named("rain");
using (Variable.If(cloudy))
{
    rain.SetTo(Variable.Bernoulli(0.8));  // Tres probable si nuageux
}
using (Variable.IfNot(cloudy))
{
    rain.SetTo(Variable.Bernoulli(0.2));  // Peu probable si ensoleille
}

// WetGrass conditionne par Sprinkler ET Rain
Variable<bool> wetGrass = Variable.New<bool>().Named("wetGrass");

// CPT pour WetGrass
// S=F, R=F -> P(W=T) = 0.0
// S=F, R=T -> P(W=T) = 0.9
// S=T, R=F -> P(W=T) = 0.9
// S=T, R=T -> P(W=T) = 0.99

using (Variable.If(sprinkler))
{
    using (Variable.If(rain))
    {
        wetGrass.SetTo(Variable.Bernoulli(0.99));  // S=T, R=T
    }
    using (Variable.IfNot(rain))
    {
        wetGrass.SetTo(Variable.Bernoulli(0.9));   // S=T, R=F
    }
}
using (Variable.IfNot(sprinkler))
{
    using (Variable.If(rain))
    {
        wetGrass.SetTo(Variable.Bernoulli(0.9));   // S=F, R=T
    }
    using (Variable.IfNot(rain))
    {
        wetGrass.SetTo(Variable.Bernoulli(0.0));   // S=F, R=F
    }
}

Console.WriteLine("Reseau Wet Grass defini.");

Reseau Wet Grass defini.


### Analyse de la structure du modele

Le modele Wet Grass est maintenant defini. Observons les elements cles de l'implementation :

| Element Infer.NET | Utilisation | Signification |
|-------------------|-------------|---------------|
| `Variable.Bernoulli(p)` | Prior marginal | Variable racine avec probabilite p |
| `Variable.New<bool>()` | Variable conditionnelle | Declaration sans prior (defini ensuite) |
| `Variable.If(x)` / `Variable.IfNot(x)` | Branchement conditionnel | Encode les CPT via des blocs imbriques |
| `SetTo()` | Affectation conditionnelle | Definit la distribution dans chaque branche |

**Structure du graphe de facteurs** : Infer.NET compile ce code en un graphe de facteurs equivalent :
- 4 variables : Cloudy, Sprinkler, Rain, WetGrass
- 4 facteurs : 1 prior (Cloudy) + 3 CPT conditionnelles

> **Point technique** : Les blocs `Variable.If`/`Variable.IfNot` imbriques implementent naturellement les tables de probabilites conditionnelles. Chaque combinaison de conditions definit une entree de la CPT.

---

### Inference des probabilites marginales

Nous allons maintenant utiliser le moteur d'inference pour calculer les **probabilites marginales** de chaque variable, c'est-a-dire P(X) sans aucune observation.

L'algorithme **Expectation Propagation** (EP) propage des messages dans le graphe de facteurs pour calculer ces marginales efficacement, sans enumerer toutes les combinaisons possibles.

In [4]:
// Inference sans observations
InferenceEngine moteur = new InferenceEngine();
moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;
moteur.ShowFactorGraph = true;  // Genere le graphe de facteurs

Console.WriteLine("=== Probabilites marginales (sans observation) ===");
Console.WriteLine($"P(Cloudy) = {moteur.Infer<Bernoulli>(cloudy).GetProbTrue():F3}");
Console.WriteLine($"P(Sprinkler) = {moteur.Infer<Bernoulli>(sprinkler).GetProbTrue():F3}");
Console.WriteLine($"P(Rain) = {moteur.Infer<Bernoulli>(rain).GetProbTrue():F3}");
Console.WriteLine($"P(WetGrass) = {moteur.Infer<Bernoulli>(wetGrass).GetProbTrue():F3}");

=== Probabilites marginales (sans observation) ===
Compiling model...done.
P(Cloudy) = 0,500
P(Sprinkler) = 0,300
P(Rain) = 0,500
P(WetGrass) = 0,598


In [5]:
// Visualisation du graphe de facteurs du reseau Wet Grass
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_10_32_85.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 Bernoulli(0,5) 
 
<!-- node1 -->
 
 node1 
 
 Random 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 dist 
 
<!-- node2 -->
 
 node2 
 
 cloudy 
 
<!-- node1->node2 -->
 
 node1->node2 
 
 
 
<!-- node5 -->
 
 node5 
 
 sprinkler 
 
<!-- node2->node5 -->
 
 node2->node5 
 
 
 condition 
 
<!-- node10 -->
 
 node10 
 
 rain 
 
<!-- node2->node10 -->
 
 node2->node10 
 
 
 condition 
 
<!-- node3 -->
 
 node3 
 
 Bernoulli(0,1) 
 
<!-- node4 -->
 
 node4 
 
 Random 
 
<!-- node3->node4 -->
 
 node3->node4 
 
 
 dist 
 
<!-- node4->node5 -->
 
 node4->node5 
 
 
 
<!-- node15 -->
 
 node15 
 
 wetGrass 
 
<!-- node5->node15 -->
 
 node5->node15 
 
 
 condition 
 
<!-- node6 -->
 
 node6 
 
 Bernoulli(0,5) 
 
<!-- node7 -->
 
 node7 
 
 Random 
 
<!-- node6->node7 -->
 
 node6->node7 
 
 
 dist 
 
<!-- node7->node5 -->
 
 node7->node5 
 
 
 
<!-- node8 -->
 
 node8 
 
 Bernoulli(0,8) 
 
<!-- node9 -->
 
 node9 
 
 Random 
 
<!-- node8->node9 -->
 
 node8->node9 
 
 
 dist 
 
<!-- node9->node10 -->
 
 node9->node10 
 
 
 
<!-- node10->node15 -->
 
 node10->node15 
 
 
 condition 
 
<!-- node11 -->
 
 node11 
 
 Bernoulli(0,2) 
 
<!-- node12 -->
 
 node12 
 
 Random 
 
<!-- node11->node12 -->
 
 node11->node12 
 
 
 dist 
 
<!-- node12->node10 -->
 
 node12->node10 
 
 
 
<!-- node13 -->
 
 node13 
 
 Bernoulli(0,99) 
 
<!-- node14 -->
 
 node14 
 
 Random 
 
<!-- node13->node14 -->
 
 node13->node14 
 
 
 dist 
 
<!-- node14->node15 -->
 
 node14->node15 
 
 
 
<!-- node16 -->
 
 node16 
 
 Bernoulli(0,9) 
 
<!-- node17 -->
 
 node17 
 
 Random 
 
<!-- node16->node17 -->
 
 node16->node17 
 
 
 dist 
 
<!-- node17->node15 -->
 
 node17->node15 
 
 
 
<!-- node18 -->
 
 node18 
 
 Bernoulli(0,9) 
 
<!-- node19 -->
 
 node19 
 
 Random 
 
<!-- node18->node19 -->
 
 node18->node19 
 
 
 dist 
 
<!-- node19->node15 -->
 
 node19->node15 
 
 
 
<!-- node20 -->
 
 node20 
 
 Bernoulli(0) 
 
<!-- node21 -->
 
 node21 
 
 Random 
 
<!-- node20->node21 -->
 
 node20->node21 
 
 
 dist 
 
<!-- node21->node15 -->
 
 node21->node15


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Lecture du graphe de facteurs

Le graphe ci-dessus represente la structure compilee par Infer.NET :

- **Cercles/Ovales** : Variables aleatoires (Cloudy, Sprinkler, Rain, WetGrass)
- **Carres/Rectangles** : Facteurs (distributions conditionnelles, CPT)
- **Aretes** : Connexions variable-facteur

La structure reflète le DAG bayésien original avec ses 4 noeuds, mais Infer.NET le compile en graphe de facteurs où les CPT deviennent des facteurs explicites. Notez comment les blocs `Variable.If/IfNot` imbriqués sont représentés.

### Analyse des probabilites marginales

**Résultats** :
- P(Cloudy) = 0.500 — le prior direct
- P(Sprinkler) = 0.300 — moyenne pondérée : 0.5×0.1 + 0.5×0.5 = 0.30 ✓
- P(Rain) = 0.500 — moyenne pondérée : 0.5×0.8 + 0.5×0.2 = 0.50 ✓
- P(WetGrass) = 0.598 — plus complexe, dépend des 4 combinaisons S×R

**Calcul détaillé de P(WetGrass)** :

$$P(W) = \sum_{s,r} P(W|S=s,R=r) \cdot P(S=s) \cdot P(R=r)$$

La valeur ~60% reflète que l'herbe mouillée est un phénomène relativement fréquent avec ces paramètres.

---

### Vers l'inference conditionnelle

Les marginales nous donnent les probabilites "a priori" (sans observation). Le veritable pouvoir des reseaux bayesiens reside dans leur capacite a **mettre a jour ces croyances** lorsqu'on observe des evidences.

Que se passe-t-il quand on observe que **l'herbe est mouillee** ? Les probabilites des causes potentielles (pluie, arroseur) doivent etre revisees selon le theoreme de Bayes.

## 4. Inference avec Observations

In [6]:
// Nouveau modele pour inference avec observation
Variable<bool> cloudy2 = Variable.Bernoulli(0.5).Named("cloudy2");
Variable<bool> sprinkler2 = Variable.New<bool>().Named("sprinkler2");
Variable<bool> rain2 = Variable.New<bool>().Named("rain2");
Variable<bool> wetGrass2 = Variable.New<bool>().Named("wetGrass2");

// Meme structure que precedemment
using (Variable.If(cloudy2))
{
    sprinkler2.SetTo(Variable.Bernoulli(0.1));
    rain2.SetTo(Variable.Bernoulli(0.8));
}
using (Variable.IfNot(cloudy2))
{
    sprinkler2.SetTo(Variable.Bernoulli(0.5));
    rain2.SetTo(Variable.Bernoulli(0.2));
}

using (Variable.If(sprinkler2))
{
    using (Variable.If(rain2))
        wetGrass2.SetTo(Variable.Bernoulli(0.99));
    using (Variable.IfNot(rain2))
        wetGrass2.SetTo(Variable.Bernoulli(0.9));
}
using (Variable.IfNot(sprinkler2))
{
    using (Variable.If(rain2))
        wetGrass2.SetTo(Variable.Bernoulli(0.9));
    using (Variable.IfNot(rain2))
        wetGrass2.SetTo(Variable.Bernoulli(0.0));
}

// OBSERVATION : L'herbe est mouillee
wetGrass2.ObservedValue = true;

InferenceEngine moteur2 = new InferenceEngine();
moteur2.Compiler.CompilerChoice = CompilerChoice.Roslyn;
moteur2.ShowFactorGraph = true;  // Genere le graphe avec observation

Console.WriteLine("=== Inference : P(X | WetGrass=True) ===");
Console.WriteLine($"P(Rain | WetGrass) = {moteur2.Infer<Bernoulli>(rain2).GetProbTrue():F3}");
Console.WriteLine($"P(Sprinkler | WetGrass) = {moteur2.Infer<Bernoulli>(sprinkler2).GetProbTrue():F3}");
Console.WriteLine($"P(Cloudy | WetGrass) = {moteur2.Infer<Bernoulli>(cloudy2).GetProbTrue():F3}");

=== Inference : P(X | WetGrass=True) ===
Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
Iterating: 
.........|.........|.........|.........|.........| 50
P(Rain | WetGrass) = 0,784
P(Sprinkler | WetGrass) = 0,404
P(Cloudy | WetGrass) = 0,604


In [7]:
// Visualisation du graphe avec observation WetGrass=True
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_10_34_64.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 Bernoulli(0,8) 
 
<!-- node1 -->
 
 node1 
 
 Random 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 dist 
 
<!-- node2 -->
 
 node2 
 
 rain2 
 
<!-- node1->node2 -->
 
 node1->node2 
 
 
 
<!-- node8 -->
 
 node8 
 
 True 
 
<!-- node2->node8 -->
 
 node2->node8 
 
 
 condition 
 
<!-- node3 -->
 
 node3 
 
 cloudy2 
 
<!-- node3->node2 -->
 
 node3->node2 
 
 
 condition 
 
<!-- node9 -->
 
 node9 
 
 sprinkler2 
 
<!-- node3->node9 -->
 
 node3->node9 
 
 
 condition 
 
<!-- node4 -->
 
 node4 
 
 Bernoulli(0,2) 
 
<!-- node5 -->
 
 node5 
 
 Random 
 
<!-- node4->node5 -->
 
 node4->node5 
 
 
 dist 
 
<!-- node5->node2 -->
 
 node5->node2 
 
 
 
<!-- node6 -->
 
 node6 
 
 Bernoulli(0,99) 
 
<!-- node7 -->
 
 node7 
 
 Random 
 
<!-- node6->node7 -->
 
 node6->node7 
 
 
 dist 
 
<!-- node7->node8 -->
 
 node7->node8 
 
 
 
<!-- node9->node8 -->
 
 node9->node8 
 
 
 condition 
 
<!-- node10 -->
 
 node10 
 
 Bernoulli(0,9) 
 
<!-- node11 -->
 
 node11 
 
 Random 
 
<!-- node10->node11 -->
 
 node10->node11 
 
 
 dist 
 
<!-- node11->node8 -->
 
 node11->node8 
 
 
 
<!-- node12 -->
 
 node12 
 
 Bernoulli(0,9) 
 
<!-- node13 -->
 
 node13 
 
 Random 
 
<!-- node12->node13 -->
 
 node12->node13 
 
 
 dist 
 
<!-- node13->node8 -->
 
 node13->node8 
 
 
 
<!-- node14 -->
 
 node14 
 
 Bernoulli(0) 
 
<!-- node15 -->
 
 node15 
 
 Random 
 
<!-- node14->node15 -->
 
 node14->node15 
 
 
 dist 
 
<!-- node15->node8 -->
 
 node15->node8 
 
 
 
<!-- node16 -->
 
 node16 
 
 Bernoulli(0,1) 
 
<!-- node17 -->
 
 node17 
 
 Random 
 
<!-- node16->node17 -->
 
 node16->node17 
 
 
 dist 
 
<!-- node17->node9 -->
 
 node17->node9 
 
 
 
<!-- node18 -->
 
 node18 
 
 Bernoulli(0,5) 
 
<!-- node19 -->
 
 node19 
 
 Random 
 
<!-- node18->node19 -->
 
 node18->node19 
 
 
 dist 
 
<!-- node19->node9 -->
 
 node19->node9 
 
 
 
<!-- node20 -->
 
 node20 
 
 Bernoulli(0,5) 
 
<!-- node21 -->
 
 node21 
 
 Random 
 
<!-- node20->node21 -->
 
 node20->node21 
 
 
 dist 
 
<!-- node21->node3 -->
 
 node21->node3


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Impact de l'observation sur le graphe

Comparez ce graphe avec le precedent. L'observation `WetGrass=True` est integree directement dans la structure du modele compile. Dans Infer.NET, une variable observee n'est plus une variable aleatoire mais une constante, ce qui simplifie certains facteurs.

L'inference remonte maintenant des effets vers les causes (**abduction**), mettant a jour les probabilites de Rain, Sprinkler et Cloudy.

### Analyse de l'inférence conditionnelle

**Résultats** : P(X | WetGrass=True)

| Variable | Prior | Posterior | Changement |
|----------|-------|-----------|------------|
| Rain | 0.500 | **0.784** | +57% |
| Sprinkler | 0.300 | **0.404** | +35% |
| Cloudy | 0.500 | **0.604** | +21% |

**Interprétation** :
- L'herbe mouillée est une **forte évidence** pour la pluie (car P(W|R,¬S)=0.9)
- L'arroseur est aussi plus probable, mais moins fortement
- Le temps nuageux augmente car il favorise la pluie, qui explique bien l'herbe mouillée

> **Raisonnement abductif** : L'inférence "remonte" des effets vers les causes. C'est le cœur du diagnostic bayésien.

### Phenomene de l'Explaining Away

Le phenomene d'**explaining away** est central en raisonnement bayesien. Il se produit lorsque deux causes alternatives d'un meme effet "competent" pour l'expliquer.

**Scenario** : L'herbe est mouillee. Deux causes possibles : pluie ou arroseur.
- Si on observe seulement l'herbe mouillee, les deux causes deviennent plus probables
- Mais si on apprend EN PLUS qu'il a plu, l'arroseur devient MOINS necessaire comme explication

Verifions ce phenomene en ajoutant l'observation Rain=True au modele precedent.

In [8]:
// Demonstation de l'Explaining Away
Variable<bool> cloudy3 = Variable.Bernoulli(0.5);
Variable<bool> sprinkler3 = Variable.New<bool>();
Variable<bool> rain3 = Variable.New<bool>();
Variable<bool> wetGrass3 = Variable.New<bool>();

using (Variable.If(cloudy3))
{
    sprinkler3.SetTo(Variable.Bernoulli(0.1));
    rain3.SetTo(Variable.Bernoulli(0.8));
}
using (Variable.IfNot(cloudy3))
{
    sprinkler3.SetTo(Variable.Bernoulli(0.5));
    rain3.SetTo(Variable.Bernoulli(0.2));
}

using (Variable.If(sprinkler3))
{
    using (Variable.If(rain3))
        wetGrass3.SetTo(Variable.Bernoulli(0.99));
    using (Variable.IfNot(rain3))
        wetGrass3.SetTo(Variable.Bernoulli(0.9));
}
using (Variable.IfNot(sprinkler3))
{
    using (Variable.If(rain3))
        wetGrass3.SetTo(Variable.Bernoulli(0.9));
    using (Variable.IfNot(rain3))
        wetGrass3.SetTo(Variable.Bernoulli(0.0));
}

// Observations : WetGrass=True ET Rain=True
wetGrass3.ObservedValue = true;
rain3.ObservedValue = true;

InferenceEngine moteur3 = new InferenceEngine();
moteur3.Compiler.CompilerChoice = CompilerChoice.Roslyn;
moteur3.ShowFactorGraph = true;  // Graphe avec deux observations

Console.WriteLine("=== Explaining Away ===");
Console.WriteLine($"P(Sprinkler | WetGrass, Rain) = {moteur3.Infer<Bernoulli>(sprinkler3).GetProbTrue():F3}");
Console.WriteLine("\nComparaison :");
Console.WriteLine("  P(Sprinkler | WetGrass) ~ 0.43");
Console.WriteLine("  P(Sprinkler | WetGrass, Rain) ~ 0.19");
Console.WriteLine("\n=> La pluie 'explique' l'herbe mouillee, reduisant P(Sprinkler)");

=== Explaining Away ===
Compiling model...done.
P(Sprinkler | WetGrass, Rain) = 0,194

Comparaison :
  P(Sprinkler | WetGrass) ~ 0.43
  P(Sprinkler | WetGrass, Rain) ~ 0.19

=> La pluie 'explique' l'herbe mouillee, reduisant P(Sprinkler)


In [9]:
// Visualisation du graphe avec Explaining Away (Rain et WetGrass observes)
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_10_35_50.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 Bernoulli(0,1) 
 
<!-- node1 -->
 
 node1 
 
 Random 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 dist 
 
<!-- node2 -->
 
 node2 
 
 vbool25 
 
<!-- node1->node2 -->
 
 node1->node2 
 
 
 
<!-- node8 -->
 
 node8 
 
 True 
 
<!-- node2->node8 -->
 
 node2->node8 
 
 
 condition 
 
<!-- node3 -->
 
 node3 
 
 vbool24 
 
<!-- node3->node2 -->
 
 node3->node2 
 
 
 condition 
 
<!-- node9 -->
 
 node9 
 
 True 
 
<!-- node3->node9 -->
 
 node3->node9 
 
 
 condition 
 
<!-- node4 -->
 
 node4 
 
 Bernoulli(0,5) 
 
<!-- node5 -->
 
 node5 
 
 Random 
 
<!-- node4->node5 -->
 
 node4->node5 
 
 
 dist 
 
<!-- node5->node2 -->
 
 node5->node2 
 
 
 
<!-- node6 -->
 
 node6 
 
 Bernoulli(0,99) 
 
<!-- node7 -->
 
 node7 
 
 Random 
 
<!-- node6->node7 -->
 
 node6->node7 
 
 
 dist 
 
<!-- node7->node8 -->
 
 node7->node8 
 
 
 
<!-- node9->node8 -->
 
 node9->node8 
 
 
 condition 
 
<!-- node10 -->
 
 node10 
 
 Bernoulli(0,9) 
 
<!-- node11 -->
 
 node11 
 
 Random 
 
<!-- node10->node11 -->
 
 node10->node11 
 
 
 dist 
 
<!-- node11->node8 -->
 
 node11->node8 
 
 
 
<!-- node12 -->
 
 node12 
 
 Bernoulli(0,9) 
 
<!-- node13 -->
 
 node13 
 
 Random 
 
<!-- node12->node13 -->
 
 node12->node13 
 
 
 dist 
 
<!-- node13->node8 -->
 
 node13->node8 
 
 
 
<!-- node14 -->
 
 node14 
 
 Bernoulli(0) 
 
<!-- node15 -->
 
 node15 
 
 Random 
 
<!-- node14->node15 -->
 
 node14->node15 
 
 
 dist 
 
<!-- node15->node8 -->
 
 node15->node8 
 
 
 
<!-- node16 -->
 
 node16 
 
 Bernoulli(0,8) 
 
<!-- node17 -->
 
 node17 
 
 Random 
 
<!-- node16->node17 -->
 
 node16->node17 
 
 
 dist 
 
<!-- node17->node9 -->
 
 node17->node9 
 
 
 
<!-- node18 -->
 
 node18 
 
 Bernoulli(0,2) 
 
<!-- node19 -->
 
 node19 
 
 Random 
 
<!-- node18->node19 -->
 
 node18->node19 
 
 
 dist 
 
<!-- node19->node9 -->
 
 node19->node9 
 
 
 
<!-- node20 -->
 
 node20 
 
 Bernoulli(0,5) 
 
<!-- node21 -->
 
 node21 
 
 Random 
 
<!-- node20->node21 -->
 
 node20->node21 
 
 
 dist 
 
<!-- node21->node3 -->
 
 node21->node3


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Structure de l'Explaining Away dans le graphe

Dans ce graphe, deux variables sont observees (Rain=True, WetGrass=True). Observez comment le graphe se simplifie :

- **Rain** n'est plus une variable aleatoire mais une constante
- **WetGrass** egalement, ce qui "coupe" certains chemins d'inference

Le phenomene d'explaining away emerge de la structure de **collider** au niveau de WetGrass. Les deux causes (Sprinkler et Rain) etaient independantes marginalement, mais deviennent dependantes une fois l'effet (WetGrass) observe.

### Explication du phénomène "Explaining Away"

**Résultat clé** : P(Sprinkler | WetGrass, Rain) = **0.194** vs P(Sprinkler | WetGrass) ≈ 0.40

Quand on observe que **l'herbe est mouillée ET qu'il a plu**, la pluie "explique" suffisamment l'observation. L'arroseur devient **moins nécessaire** comme explication.

**Analogie intuitive** : Si vous arrivez au bureau trempé et que je sais qu'il pleut dehors, je ne vais pas penser que vous êtes aussi passé sous un arroseur. Une explication suffit.

**Structure graphique** :
```
(Sprinkler) → (WetGrass) ← (Rain)
            [collider]
```

Au collider, les deux causes sont **marginalement indépendantes** mais deviennent **conditionnellement dépendantes** quand l'effet est observé. C'est contre-intuitif mais fondamental en raisonnement bayésien.

## 5. D-Separation et Independance Conditionnelle

### Definition

La **D-separation** est un critere graphique pour determiner l'independance conditionnelle dans un reseau bayesien.

### Trois structures de base

| Structure | Nom | Independance |
|-----------|-----|-------------|
| A -> B -> C | Chaine | A _|_ C \| B |
| A <- B -> C | Fork (cause commune) | A _|_ C \| B |
| A -> B <- C | Collider (effet commun) | A _|_ C, mais A /|/ C \| B |

### Application au reseau Wet Grass

Le reseau Wet Grass contient deux structures cles :

1. **Fork** (S <- C -> R) : Sprinkler et Rain ont une cause commune (Cloudy)
   - Marginalement : S et R sont **dependants** (l'info passe par C)
   - Conditionnellement a C : S et R sont **independants**

2. **Collider** (S -> W <- R) : WetGrass est un effet commun
   - Marginalement : S et R sont **independants** via ce chemin
   - Conditionnellement a W : S et R deviennent **dependants** (explaining away)

In [10]:
// Verification de la D-separation

// Test 1 : Sprinkler et Rain sans observation sur WetGrass
Variable<bool> c4 = Variable.Bernoulli(0.5);
Variable<bool> s4 = Variable.New<bool>();
Variable<bool> r4 = Variable.New<bool>();

using (Variable.If(c4)) { s4.SetTo(Variable.Bernoulli(0.1)); r4.SetTo(Variable.Bernoulli(0.8)); }
using (Variable.IfNot(c4)) { s4.SetTo(Variable.Bernoulli(0.5)); r4.SetTo(Variable.Bernoulli(0.2)); }

// Observation : Sprinkler = true
s4.ObservedValue = true;

InferenceEngine m4 = new InferenceEngine();
m4.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Console.WriteLine("=== Test D-separation (sans WetGrass) ===");
Console.WriteLine($"P(Rain | Sprinkler) = {m4.Infer<Bernoulli>(r4).GetProbTrue():F3}");
Console.WriteLine("P(Rain) marginale = 0.500");
Console.WriteLine("\n=> S et R ne sont PAS marginalement independants !");
Console.WriteLine("   Observer S donne de l'information sur C, qui informe R.");

=== Test D-separation (sans WetGrass) ===
Compiling model...done.
P(Rain | Sprinkler) = 0,300
P(Rain) marginale = 0.500

=> S et R ne sont PAS marginalement independants !
   Observer S donne de l'information sur C, qui informe R.


### Analyse du resultat

**Resultat** : P(Rain | Sprinkler=True) = **0.300** != P(Rain) = 0.500

Ceci confirme que S et R sont **marginalement dependants** dans la structure fork.

**Explication** : Observer Sprinkler=True donne de l'information sur Cloudy via le theoreme de Bayes :

$$P(\text{Cloudy} | S=T) = \frac{P(S=T|\text{Cloudy}) \cdot P(\text{Cloudy})}{P(S=T)} = \frac{0.1 \times 0.5}{0.3} \approx 0.167$$

Donc il fait probablement beau (ensoleille), et :

$$P(\text{Rain} | S=T) \approx P(R|C) \cdot P(C|S) + P(R|\neg C) \cdot P(\neg C|S) = 0.8 \times 0.167 + 0.2 \times 0.833 \approx 0.30$$

> **Point cle** : Dans un fork, les variables enfants sont independantes **conditionnellement au parent**, pas marginalement. L'information "passe" par la cause commune.

---

### Verification de l'independance conditionnelle

Pour verifier la D-separation, testons ce qui se passe quand on **observe Cloudy** en plus de Sprinkler. Si la theorie est correcte, Rain ne devrait plus changer.

In [11]:
// Test 2 : D-separation avec Cloudy observe
// Si on conditionne sur Cloudy, S et R deviennent independants

Variable<bool> c5 = Variable.Bernoulli(0.5);
Variable<bool> s5 = Variable.New<bool>();
Variable<bool> r5 = Variable.New<bool>();

using (Variable.If(c5)) { s5.SetTo(Variable.Bernoulli(0.1)); r5.SetTo(Variable.Bernoulli(0.8)); }
using (Variable.IfNot(c5)) { s5.SetTo(Variable.Bernoulli(0.5)); r5.SetTo(Variable.Bernoulli(0.2)); }

// Observations : Cloudy = false ET Sprinkler = true
c5.ObservedValue = false;  // On fixe Cloudy
s5.ObservedValue = true;

InferenceEngine m5 = new InferenceEngine();
m5.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Console.WriteLine("=== Test D-separation (avec Cloudy observe) ===");
Console.WriteLine($"P(Rain | Sprinkler, Cloudy=False) = {m5.Infer<Bernoulli>(r5).GetProbTrue():F3}");
Console.WriteLine("P(Rain | Cloudy=False) = 0.200");
Console.WriteLine("\n=> S et R sont independants conditionnellement a C !");
Console.WriteLine("   Une fois C connu, observer S n'apporte plus d'information sur R.");

=== Test D-separation (avec Cloudy observe) ===
Compiling model...done.
P(Rain | Sprinkler, Cloudy=False) = 0,200
P(Rain | Cloudy=False) = 0.200

=> S et R sont independants conditionnellement a C !
   Une fois C connu, observer S n'apporte plus d'information sur R.


**Verification de l'independance conditionnelle** :

Le resultat P(Rain | S, C=F) = **0.200** = P(Rain | C=F) confirme que :

$$S \perp\!\!\!\perp R \mid C$$

Sprinkler et Rain sont **conditionnellement independants** sachant Cloudy. C'est exactement ce que predit la D-separation pour une structure fork.

| Test | Resultat | Interpretation |
|------|----------|----------------|
| P(R \| S) = 0.30 | != P(R) = 0.50 | S et R dependants marginalement |
| P(R \| S, C) = 0.20 | = P(R \| C) = 0.20 | S et R independants sachant C |

## 6. Inference Causale vs Observationnelle

### Difference fondamentale

| Type | Question | Notation |
|------|----------|----------|
| **Observationnel** | P(Y\|X=x) | Que se passe-t-il quand on observe X=x ? |
| **Interventionnel** | P(Y\|do(X=x)) | Que se passe-t-il quand on force X=x ? |

### Exemple

- **Observer** qu'il pleut : P(WetGrass | Rain=True)
- **Faire pleuvoir** artificiellement : P(WetGrass | do(Rain=True))

Dans le second cas, on "coupe" les arcs entrants vers Rain (il ne depend plus de Cloudy).

---

### Implementation de l'operateur do()

Pour simuler une **intervention** (operateur do()), on modifie le modele en :
1. **Coupant** les arcs entrants vers la variable manipulee
2. **Fixant** sa valeur a la valeur d'intervention

Comparons P(Cloudy | Rain=True) (observation) avec P(Cloudy | do(Rain=True)) (intervention).

In [12]:
// Comparaison inference observationnelle vs interventionnelle

// OBSERVATIONNEL : P(Cloudy | Rain=True)
Variable<bool> cObs = Variable.Bernoulli(0.5);
Variable<bool> rObs = Variable.New<bool>();

using (Variable.If(cObs)) { rObs.SetTo(Variable.Bernoulli(0.8)); }
using (Variable.IfNot(cObs)) { rObs.SetTo(Variable.Bernoulli(0.2)); }

rObs.ObservedValue = true;

InferenceEngine mObs = new InferenceEngine();
mObs.Compiler.CompilerChoice = CompilerChoice.Roslyn;

double pCloudyObs = mObs.Infer<Bernoulli>(cObs).GetProbTrue();

// INTERVENTIONNEL : P(Cloudy | do(Rain=True))
// On "coupe" l'arc Cloudy -> Rain en fixant Rain independamment
Variable<bool> cInt = Variable.Bernoulli(0.5);
Variable<bool> rInt = Variable.Bernoulli(1.0);  // Force a True, independant de Cloudy

InferenceEngine mInt = new InferenceEngine();
mInt.Compiler.CompilerChoice = CompilerChoice.Roslyn;

double pCloudyInt = mInt.Infer<Bernoulli>(cInt).GetProbTrue();

Console.WriteLine("=== Observationnel vs Interventionnel ===");
Console.WriteLine($"P(Cloudy | Rain=True) = {pCloudyObs:F3} (observationnel)");
Console.WriteLine($"P(Cloudy | do(Rain=True)) = {pCloudyInt:F3} (interventionnel)");
Console.WriteLine("\n=> Observer la pluie informe sur le temps nuageux");
Console.WriteLine("   Faire pleuvoir ne change pas le temps nuageux");

Compiling model...done.
Compiling model...done.
=== Observationnel vs Interventionnel ===
P(Cloudy | Rain=True) = 0,800 (observationnel)
P(Cloudy | do(Rain=True)) = 0,500 (interventionnel)

=> Observer la pluie informe sur le temps nuageux
   Faire pleuvoir ne change pas le temps nuageux


### Analyse : Causalité vs Corrélation

**Résultats** :
- P(Cloudy | Rain=True) = **0.800** — observationnel
- P(Cloudy | do(Rain=True)) = **0.500** — interventionnel

**Différence fondamentale** :

| Type | Sémantique | Résultat |
|------|------------|----------|
| Observation | "Je vois qu'il pleut" | Le temps est probablement nuageux |
| Intervention | "Je fais pleuvoir (machine à pluie)" | Le temps reste incertain |

L'**opérateur do()** de Pearl "coupe" les arcs entrants vers la variable manipulée. Rain ne dépend plus de Cloudy dans le graphe modifié.

> **Application pratique** : Cette distinction est cruciale en médecine. Observer une corrélation entre un traitement et la guérison n'implique pas que le traitement cause la guérison (biais de sélection possible). Un essai randomisé (intervention) est nécessaire pour établir la causalité.

## 7. Modele Causal : Direction de la Causalite

### Question

Etant donne des donnees observationnelles entre A et B, comment determiner si A cause B ou B cause A ?

### Approche bayesienne

Comparer les evidences des deux modeles :
- $M_1$ : A -> B
- $M_2$ : B -> A

In [13]:
// Modele pour identifier la direction causale

// Donnees simulees : A cause B avec bruit
double[] dataA = { 1.0, 2.0, 3.0, 4.0, 5.0 };
double[] dataB = { 2.1, 4.2, 5.8, 8.1, 10.3 };  // B ~ 2*A + bruit

int n = dataA.Length;

// Modele 1 : A -> B (A cause B)
Variable<bool> evidence1 = Variable.Bernoulli(0.5);
using (Variable.If(evidence1))
{
    Variable<double> slope1 = Variable.GaussianFromMeanAndPrecision(0, 0.1);
    Variable<double> intercept1 = Variable.GaussianFromMeanAndPrecision(0, 0.1);
    Variable<double> noise1 = Variable.GammaFromShapeAndScale(2, 0.5);
    
    for (int i = 0; i < n; i++)
    {
        Variable<double> aPrior1 = Variable.GaussianFromMeanAndPrecision(0, 0.1);
        aPrior1.ObservedValue = dataA[i];
        Variable<double> bPred1 = slope1 * aPrior1 + intercept1;
        Variable<double> bObs1 = Variable.GaussianFromMeanAndPrecision(bPred1, noise1);
        bObs1.ObservedValue = dataB[i];
    }
}

InferenceEngine mCausal1 = new InferenceEngine();
mCausal1.Compiler.CompilerChoice = CompilerChoice.Roslyn;

double logEvidence1 = mCausal1.Infer<Bernoulli>(evidence1).LogOdds;

Console.WriteLine("=== Selection de la direction causale ===");
Console.WriteLine($"Log Evidence (A -> B) = {logEvidence1:F3}");
Console.WriteLine("\nNote: Un modele plus sophistique comparerait avec B -> A");

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
=== Selection de la direction causale ===
Log Evidence (A -> B) = -22,920

Note: Un modele plus sophistique comparerait avec B -> A


### Interpretation du log-evidence

**Resultat** : Log Evidence (A -> B) = -22.920

Cette valeur represente le **log de la vraisemblance marginale** du modele. Pour comparer deux modeles :

$$\text{Bayes Factor} = \frac{P(\text{Data} | M_1)}{P(\text{Data} | M_2)} = \exp(\log E_1 - \log E_2)$$

**Echelle d'interpretation du Bayes Factor** (Kass & Raftery, 1995) :

| log(BF) | BF | Interpretation |
|---------|-----|----------------|
| < 1 | < 3 | Evidence negligeable |
| 1-3 | 3-20 | Evidence positive |
| 3-5 | 20-150 | Evidence forte |
| > 5 | > 150 | Evidence tres forte |

> **Limitation** : Cette approche suppose des modeles lineaires gaussiens. Pour des relations non-lineaires ou des variables discretes, des methodes comme PC-algorithm ou GES sont plus appropriees.

---

### Vers les modeles hierarchiques

Les modeles hierarchiques (ou multiniveaux) etendent les reseaux bayesiens en introduisant des **hyperparametres** qui gouvernent les distributions des parametres individuels. Cette structure permet de modeliser la variabilite entre groupes ou individus.

## 8. Modele BUGS Rats (Modele Hierarchique)

### Contexte

Le modele "Rats" de BUGS est un exemple classique de modele hierarchique :
- Plusieurs rats suivis dans le temps
- Chaque rat a sa propre trajectoire de croissance
- Les parametres individuels sont tires d'une distribution de population

### Structure hierarchique

```
Population: mu_alpha, sigma_alpha, mu_beta, sigma_beta
     |
     v
Individu: alpha[i], beta[i] ~ N(mu, sigma)
     |
     v
Observation: y[i,t] = alpha[i] + beta[i] * t + epsilon
```

---

### Implementation Infer.NET du modele hierarchique

Le modele suivant illustre la structure hierarchique avec :
- **Niveau population** : hyperparametres mu_alpha, mu_beta, tau_alpha, tau_beta
- **Niveau individuel** : parametres alpha[i], beta[i] pour chaque rat
- **Niveau observation** : mesures de poids y[i,t]

Cette structure permet le **partage d'information** entre individus via les distributions de population.

In [14]:
// Modele hierarchique simplifie (style BUGS Rats)

int nRats = 5;
int nTimePoints = 4;
double[] times = { 8.0, 15.0, 22.0, 29.0 };  // Ages en jours

// Donnees simulees : poids des rats au fil du temps
double[,] weights = new double[,] {
    { 151, 199, 246, 283 },
    { 145, 199, 249, 293 },
    { 147, 214, 263, 312 },
    { 155, 200, 237, 272 },
    { 135, 188, 230, 280 }
};

// Parametres de population
Variable<double> muAlpha = Variable.GaussianFromMeanAndVariance(150, 1000);
Variable<double> muBeta = Variable.GaussianFromMeanAndVariance(5, 100);
Variable<double> tauAlpha = Variable.GammaFromShapeAndScale(1, 1);
Variable<double> tauBeta = Variable.GammaFromShapeAndScale(1, 1);
Variable<double> tauNoise = Variable.GammaFromShapeAndScale(1, 1);

// Parametres individuels
Range ratRange = new Range(nRats);
VariableArray<double> alpha = Variable.Array<double>(ratRange);
VariableArray<double> beta = Variable.Array<double>(ratRange);

using (Variable.ForEach(ratRange))
{
    alpha[ratRange] = Variable.GaussianFromMeanAndPrecision(muAlpha, tauAlpha);
    beta[ratRange] = Variable.GaussianFromMeanAndPrecision(muBeta, tauBeta);
}

// Observations
for (int i = 0; i < nRats; i++)
{
    for (int t = 0; t < nTimePoints; t++)
    {
        Variable<double> meanWeight = alpha[i] + beta[i] * (times[t] - 22.0);
        Variable<double> obsWeight = Variable.GaussianFromMeanAndPrecision(meanWeight, tauNoise);
        obsWeight.ObservedValue = weights[i, t];
    }
}

InferenceEngine mRats = new InferenceEngine(new ExpectationPropagation());
mRats.Compiler.CompilerChoice = CompilerChoice.Roslyn;
mRats.ShowFactorGraph = true;  // Graphe du modele hierarchique

Console.WriteLine("=== Modele Hierarchique (Rats) ===");
Console.WriteLine($"mu_alpha (intercept moyen) : {mRats.Infer<Gaussian>(muAlpha)}");
Console.WriteLine($"mu_beta (pente moyenne) : {mRats.Infer<Gaussian>(muBeta)}");

=== Modele Hierarchique (Rats) ===
Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
mu_alpha (intercept moyen) : Gaussian(241,7, 22,96)
mu_beta (pente moyenne) : Gaussian(6,68, 0,2134)


In [15]:
// Visualisation du graphe hierarchique (Rats)
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_10_38_66.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 150 
 
<!-- node1 -->
 
 node1 
 
 GaussianFromMeanAndVariance 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 mean 
 
<!-- node3 -->
 
 node3 
 
 vdouble41 
 
<!-- node1->node3 -->
 
 node1->node3 
 
 
 
<!-- node2 -->
 
 node2 
 
 1000 
 
<!-- node2->node1 -->
 
 node2->node1 
 
 
 variance 
 
<!-- node4 -->
 
 node4 
 
 Gaussian 
 
<!-- node3->node4 -->
 
 node3->node4 
 
 
 mean 
 
<!-- node6 -->
 
 node6 
 
 vdouble[]0[index0] 
 
<!-- node4->node6 -->
 
 node4->node6 
 
 
 
<!-- node5 -->
 
 node5 
 
 vdouble47 
 
<!-- node5->node4 -->
 
 node5->node4 
 
 
 precision 
 
<!-- node10 -->
 
 node10 
 
 vdouble[]0[vint2] 
 
<!-- node6->node10 -->
 
 node6->node10 
 
 
<!-- node14 -->
 
 node14 
 
 vdouble[]0[vint4] 
 
<!-- node6->node14 -->
 
 node6->node14 
 
 
<!-- node18 -->
 
 node18 
 
 vdouble[]0[vint6] 
 
<!-- node6->node18 -->
 
 node6->node18 
 
 
<!-- node22 -->
 
 node22 
 
 vdouble[]0[vint8] 
 
<!-- node6->node22 -->
 
 node6->node22 
 
 
<!-- node26 -->
 
 node26 
 
 vdouble[]0[vint10] 
 
<!-- node6->node26 -->
 
 node6->node26 
 
 
<!-- node30 -->
 
 node30 
 
 vdouble[]0[vint12] 
 
<!-- node6->node30 -->
 
 node6->node30 
 
 
<!-- node34 -->
 
 node34 
 
 vdouble[]0[vint14] 
 
<!-- node6->node34 -->
 
 node6->node34 
 
 
<!-- node38 -->
 
 node38 
 
 vdouble[]0[vint16] 
 
<!-- node6->node38 -->
 
 node6->node38 
 
 
<!-- node42 -->
 
 node42 
 
 vdouble[]0[vint18] 
 
<!-- node6->node42 -->
 
 node6->node42 
 
 
<!-- node46 -->
 
 node46 
 
 vdouble[]0[vint20] 
 
<!-- node6->node46 -->
 
 node6->node46 
 
 
<!-- node50 -->
 
 node50 
 
 vdouble[]0[vint22] 
 
<!-- node6->node50 -->
 
 node6->node50 
 
 
<!-- node54 -->
 
 node54 
 
 vdouble[]0[vint24] 
 
<!-- node6->node54 -->
 
 node6->node54 
 
 
<!-- node58 -->
 
 node58 
 
 vdouble[]0[vint26] 
 
<!-- node6->node58 -->
 
 node6->node58 
 
 
<!-- node62 -->
 
 node62 
 
 vdouble[]0[vint28] 
 
<!-- node6->node62 -->
 
 node6->node62 
 
 
<!-- node66 -->
 
 node66 
 
 vdouble[]0[vint30] 
 
<!-- node6->node66 -->
 
 node6->node66 
 
 
<!-- node70 -->
 
 node70 
 
 vdouble[]0[vint32] 
 
<!-- node6->node70 -->
 
 node6->node70 
 
 
<!-- node74 -->
 
 node74 
 
 vdouble[]0[vint34] 
 
<!-- node6->node74 -->
 
 node6->node74 
 
 
<!-- node78 -->
 
 node78 
 
 vdouble[]0[vint36] 
 
<!-- node6->node78 -->
 
 node6->node78 
 
 
<!-- node82 -->
 
 node82 
 
 vdouble[]0[vint38] 
 
<!-- node6->node82 -->
 
 node6->node82 
 
 
<!-- node86 -->
 
 node86 
 
 vdouble[]0[vint40] 
 
<!-- node6->node86 -->
 
 node6->node86 
 
 
<!-- node7 -->
 
 node7 
 
 1 
 
<!-- node8 -->
 
 node8 
 
 Sample 
 
<!-- node7->node8 -->
 
 node7->node8 
 
 
 shape 
 
<!-- node8->node5 -->
 
 node8->node5 
 
 
 
<!-- node9 -->
 
 node9 
 
 1 
 
<!-- node9->node8 -->
 
 node9->node8 
 
 
 scale 
 
<!-- node11 -->
 
 node11 
 
 Plus 
 
<!-- node10->node11 -->
 
 node10->node11 
 
 
 a 
 
<!-- node13 -->
 
 node13 
 
 vdouble64 
 
<!-- node11->node13 -->
 
 node11->node13 
 
 
 
<!-- node12 -->
 
 node12 
 
 vdouble63 
 
<!-- node12->node11 -->
 
 node12->node11 
 
 
 b 
 
<!-- node166 -->
 
 node166 
 
 Gaussian 
 
<!-- node13->node166 -->
 
 node13->node166 
 
 
 mean 
 
<!-- node15 -->
 
 node15 
 
 Plus 
 
<!-- node14->node15 -->
 
 node14->node15 
 
 
 a 
 
<!-- node17 -->


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Anatomie du modele hierarchique

Le graphe illustre la structure a deux niveaux du modele Rats :

**Niveau population (hyperparametres)** :
- `muAlpha`, `muBeta` : moyennes de population
- `tauAlpha`, `tauBeta` : precisions de population

**Niveau individuel (parametres)** :
- `alpha[i]`, `beta[i]` : parametres pour chaque rat
- Les arrays sont representes par des facteurs repliques

**Niveau observation** :
- Les observations de poids sont attachees aux predictions individuelles

Cette structure **plate** est caracteristique des modeles hierarchiques compiles : les boucles `ForEach` sont deroulees en facteurs individuels partageant les memes hyperparametres.

### Analyse du modèle hiérarchique

**Résultats** :
- μ_alpha (poids moyen à t=22 jours) ≈ **241.7 g**
- μ_beta (croissance moyenne) ≈ **6.68 g/jour**

**Interprétation biologique** :
- À 22 jours, les rats pèsent en moyenne ~242g
- Ils prennent environ 6.7g par jour

**Avantages du modèle hiérarchique** :

| Aspect | Explication |
|--------|-------------|
| **Partage d'information** | Les rats "empruntent" de l'information les uns aux autres via la population |
| **Shrinkage** | Les estimations individuelles sont tirées vers la moyenne |
| **Robustesse** | Moins sensible aux outliers individuels |
| **Généralisation** | Peut prédire pour un nouveau rat non observé |

> **Concept clé** : La hiérarchie "régularise" naturellement les estimations. Un rat avec peu d'observations sera tiré vers la moyenne de population, évitant le sur-ajustement.

## 9. Exercice : Reseau de Diagnostic Medical

### Enonce

Construisez un reseau bayesien pour le diagnostic de deux maladies :

**Structure** :
```
(Cold)     (Flu)
   \         /
    v       v
   (Fatigue)
    |
    v
  (Fever)
```

**CPTs** :
- P(Cold) = 0.02
- P(Flu) = 0.01
- P(Fatigue | Cold, Flu) : voir table
- P(Fever | Fatigue) : 0.8 si fatigue, 0.05 sinon

**Question** : P(Flu | Fever=True) ?

---

### Application pratique : Systemes d'aide au diagnostic

Les reseaux bayesiens sont largement utilises en medecine pour le diagnostic assiste par ordinateur. Historiquement :

| Systeme | Annee | Domaine | Particularite |
|---------|-------|---------|---------------|
| MYCIN | 1976 | Infections bacteriennes | Regles avec facteurs de certitude |
| INTERNIST-1 | 1982 | Medecine interne | 500+ maladies, 3500 symptomes |
| QMR | 1991 | Medecine interne | Version bayesienne d'INTERNIST |
| DXplain | 1986-present | Diagnostic general | 2400+ maladies |

L'exercice suivant illustre les principes de base d'un tel systeme.

---

### Implementation du reseau de diagnostic

Construisons le reseau avec les CPT specifiees. Le raisonnement diagnostique ira des **symptomes observes** (fievre) vers les **causes probables** (maladies).

In [16]:
// EXERCICE : Reseau de diagnostic

Variable<bool> cold = Variable.Bernoulli(0.02).Named("cold");
Variable<bool> flu = Variable.Bernoulli(0.01).Named("flu");
Variable<bool> fatigue = Variable.New<bool>().Named("fatigue");
Variable<bool> fever = Variable.New<bool>().Named("fever");

// CPT pour Fatigue
// Cold=F, Flu=F -> P(Fatigue) = 0.1
// Cold=F, Flu=T -> P(Fatigue) = 0.9
// Cold=T, Flu=F -> P(Fatigue) = 0.6
// Cold=T, Flu=T -> P(Fatigue) = 0.95

using (Variable.If(cold))
{
    using (Variable.If(flu))
        fatigue.SetTo(Variable.Bernoulli(0.95));
    using (Variable.IfNot(flu))
        fatigue.SetTo(Variable.Bernoulli(0.6));
}
using (Variable.IfNot(cold))
{
    using (Variable.If(flu))
        fatigue.SetTo(Variable.Bernoulli(0.9));
    using (Variable.IfNot(flu))
        fatigue.SetTo(Variable.Bernoulli(0.1));
}

// CPT pour Fever
using (Variable.If(fatigue))
{
    fever.SetTo(Variable.Bernoulli(0.8));
}
using (Variable.IfNot(fatigue))
{
    fever.SetTo(Variable.Bernoulli(0.05));
}

// Observation
fever.ObservedValue = true;

InferenceEngine mDiag = new InferenceEngine();
mDiag.Compiler.CompilerChoice = CompilerChoice.Roslyn;
mDiag.ShowFactorGraph = true;  // Graphe du reseau de diagnostic

Console.WriteLine("=== Diagnostic Medical ===");
Console.WriteLine($"P(Flu | Fever) = {mDiag.Infer<Bernoulli>(flu).GetProbTrue():F3}");
Console.WriteLine($"P(Cold | Fever) = {mDiag.Infer<Bernoulli>(cold).GetProbTrue():F3}");
Console.WriteLine($"P(Fatigue | Fever) = {mDiag.Infer<Bernoulli>(fatigue).GetProbTrue():F3}");

=== Diagnostic Medical ===
Compiling model...done.
P(Flu | Fever) = 0,052
P(Cold | Fever) = 0,073
P(Fatigue | Fever) = 0,681


In [17]:
// Visualisation du graphe de diagnostic medical
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_10_41_55.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 Bernoulli(0,01) 
 
<!-- node1 -->
 
 node1 
 
 Random 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 dist 
 
<!-- node2 -->
 
 node2 
 
 flu 
 
<!-- node1->node2 -->
 
 node1->node2 
 
 
 
<!-- node5 -->
 
 node5 
 
 fatigue 
 
<!-- node2->node5 -->
 
 node2->node5 
 
 
 condition 
 
<!-- node3 -->
 
 node3 
 
 Bernoulli(0,95) 
 
<!-- node4 -->
 
 node4 
 
 Random 
 
<!-- node3->node4 -->
 
 node3->node4 
 
 
 dist 
 
<!-- node4->node5 -->
 
 node4->node5 
 
 
 
<!-- node17 -->
 
 node17 
 
 True 
 
<!-- node5->node17 -->
 
 node5->node17 
 
 
 condition 
 
<!-- node6 -->
 
 node6 
 
 cold 
 
<!-- node6->node5 -->
 
 node6->node5 
 
 
 condition 
 
<!-- node7 -->
 
 node7 
 
 Bernoulli(0,6) 
 
<!-- node8 -->
 
 node8 
 
 Random 
 
<!-- node7->node8 -->
 
 node7->node8 
 
 
 dist 
 
<!-- node8->node5 -->
 
 node8->node5 
 
 
 
<!-- node9 -->
 
 node9 
 
 Bernoulli(0,9) 
 
<!-- node10 -->
 
 node10 
 
 Random 
 
<!-- node9->node10 -->
 
 node9->node10 
 
 
 dist 
 
<!-- node10->node5 -->
 
 node10->node5 
 
 
 
<!-- node11 -->
 
 node11 
 
 Bernoulli(0,1) 
 
<!-- node12 -->
 
 node12 
 
 Random 
 
<!-- node11->node12 -->
 
 node11->node12 
 
 
 dist 
 
<!-- node12->node5 -->
 
 node12->node5 
 
 
 
<!-- node13 -->
 
 node13 
 
 Bernoulli(0,02) 
 
<!-- node14 -->
 
 node14 
 
 Random 
 
<!-- node13->node14 -->
 
 node13->node14 
 
 
 dist 
 
<!-- node14->node6 -->
 
 node14->node6 
 
 
 
<!-- node15 -->
 
 node15 
 
 Bernoulli(0,8) 
 
<!-- node16 -->
 
 node16 
 
 Random 
 
<!-- node15->node16 -->
 
 node15->node16 
 
 
 dist 
 
<!-- node16->node17 -->
 
 node16->node17 
 
 
 
<!-- node18 -->
 
 node18 
 
 Bernoulli(0,05) 
 
<!-- node19 -->
 
 node19 
 
 Random 
 
<!-- node18->node19 -->
 
 node18->node19 
 
 
 dist 
 
<!-- node19->node17 -->
 
 node19->node17


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Structure du reseau de diagnostic

Le graphe montre le reseau bayesien de diagnostic avec :

- **Deux racines independantes** : Cold et Flu (maladies avec faibles prevalences)
- **Variable intermediaire** : Fatigue (symptome cause par les maladies)
- **Variable observee** : Fever (symptome terminal, observee)

Cette structure en **diamant** illustre :
1. **Causes multiples** pour Fatigue (Cold et Flu sont des causes alternatives)
2. **Chaine causale** Maladie -> Fatigue -> Fever

L'inference diagnostique remonte de Fever observee vers les causes potentielles (maladies), en passant par l'intermediaire Fatigue.

### Analyse du diagnostic médical

**Résultats** : P(Maladie | Fever=True)

| Maladie | Prior | Posterior | Ratio |
|---------|-------|-----------|-------|
| Flu | 0.01 | **0.052** | ×5.2 |
| Cold | 0.02 | **0.073** | ×3.6 |

**Observations** :

1. **Explaining away partiel** : Cold et Flu sont tous deux plus probables avec la fièvre, mais moins que si l'un des deux était certain (chacun "explique partiellement" la fièvre).

2. **P(Fatigue | Fever) = 0.681** : La fièvre est fortement associée à la fatigue car P(Fever|Fatigue) = 0.8 >> P(Fever|¬Fatigue) = 0.05.

3. **Diagnostic différentiel** : Avec seulement la fièvre, il reste difficile de distinguer Cold de Flu. Des symptômes supplémentaires (courbatures, durée) seraient nécessaires.

> **Application clinique** : Ce type de réseau bayésien est utilisé dans les systèmes d'aide au diagnostic (MYCIN, DXplain). L'ajout de symptômes affine progressivement les probabilités des maladies.

### Extension : Diagnostic avec symptomes multiples

Pour ameliorer le diagnostic, on pourrait ajouter d'autres symptomes :

```
(Cold)     (Flu)
   \   \   /   /
    \   \ /   /
     v   v   v
   (Fatigue) (BodyAche)
       |         |
       v         v
    (Fever)  (Headache)
```

**Table de discrimination** :

| Symptome | Cold | Flu | Ratio diagnostique |
|----------|------|-----|-------------------|
| Fatigue legere | 60% | 90% | 1.5 |
| Fievre elevee (>39C) | 10% | 70% | 7.0 |
| Courbatures | 20% | 80% | 4.0 |
| Duree > 7 jours | 30% | 60% | 2.0 |

> **Exercice complementaire** : Etendre le modele avec le symptome "BodyAche" et observer comment P(Flu|Fever, BodyAche) evolue par rapport a P(Flu|Fever).

## 10. Resume

### Concepts cles

| Concept | Description |
|---------|-------------|
| **Reseau bayesien** | DAG avec variables et CPTs |
| **CPT** | Table de Probabilite Conditionnelle |
| **D-separation** | Critere graphique d'independance |
| **Explaining away** | Causes alternatives deviennent moins probables |
| **Observation vs Intervention** | P(Y\|X) vs P(Y\|do(X)) |
| **Modele hierarchique** | Parametres individuels tires d'une population |

### Patterns Infer.NET

| Pattern | Code | Utilisation |
|---------|------|-------------|
| Variable racine | `Variable.Bernoulli(p)` | Prior marginal |
| CPT binaire | `Variable.If(x)` + `SetTo()` | Probabilites conditionnelles |
| Observation | `x.ObservedValue = true` | Conditionner l'inference |
| Intervention (do) | `Variable.Bernoulli(1.0)` (independant) | Couper les arcs entrants |
| Boucle hierarchique | `Variable.ForEach(range)` | Modeles multiniveaux |

### Distributions utilisees

| Distribution | Parametre | Contexte |
|--------------|-----------|----------|
| Bernoulli | probabilite p | Variables booleennes |
| Gaussian | moyenne, precision | Variables continues |
| Gamma | shape, scale | Priors sur precisions (positifs) |

---

## Prochaine etape

Dans [Infer-5-Skills-IRT](Infer-5-Skills-IRT.ipynb), nous explorerons :

- Les modeles d'evaluation de competences
- Item Response Theory (IRT) avec le modele Difficulty-Ability
- Le modele DINA (Noisy-And) pour les competences multiples
- Les relations many-to-many entre competences et questions